In [1]:
import numpy as np
import data_io
from data_preprocessing import *
from implementations import *
import validation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
DATA_FILE_PREFIX = '/content/drive/My Drive/mlproject1_higgs_data/'

In [4]:
y, x, ids, cols = data_io.load_csv_data(f'{DATA_FILE_PREFIX}train.csv')

In [5]:
# Make it easier to access columns by their name
col_to_index_mapping = {col_name: index - 2 for index, col_name in enumerate(cols) if index >= 2}

In [ ]:
col_to_index_mapping

In [7]:
#pred = map_values(np.random.randint(0, 2, y.size), {0: -1, 1: 1})
pred = np.array([-1] * y.size)

In [ ]:
import metrics
# Tests metrics (fbeta_score, precision and recall might return "RuntimeWarning: invalid value encountered in long_scalars"
#     which functions take care of (these are nan cases) by returning 0)
print('Acc:', metrics.accuracy(y, pred))
print('Fbeta:', metrics.fbeta_score(y, pred))
print('Precision:', metrics.precision(y, pred))
print('Recall:', metrics.recall(y, pred))

In [9]:
def train_model(y, x):
  #w, _ = least_squares_SGD(y, x, np.zeros((x.shape[1],)), 100, gamma=0.01)
  w, loss = least_squares(y, x)
  def predict(features):
    result = features @ w
    result[np.where(result < 0)] = -1
    result[np.where(result >= 0)] = 1
    return result
  return predict

In [ ]:
tx = np.copy(x) # Recommended to copy x so it doesn't change

missing_values = tx == -999.
print(np.sum(np.sum(missing_values, axis = 1) < 6)) # Checking how many rows with less than 6 fields missing is there
tx = apply_transformation(tx, [col_to_index_mapping[key] for key in col_to_index_mapping if 'PRI_jet_num' not in key], standardize)
tx = nullify_missing_values(tx, missing_values) # standardize and normalize may change value of fields from default missing values, so it uses matrix calculated before applying transformations
tx = apply_transformation(tx, [col_to_index_mapping['PRI_jet_num']], one_hot) # recommended to use function that map input to output of same dimensions first (standardize, normalize, etc.) and then functions such as one_hot
tx = prepend_bias_column(tx)

print(tx) 


In [ ]:
_ = validation.cross_validation(y, tx, train_model, 10, verbose=True)